In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-classification-nlp/SMS_train.csv
/kaggle/input/email-classification-nlp/SMS_test.csv


In [2]:
# Import and EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_train.csv", encoding='latin-1')
df

S. No.                                       Message_body     Label
0         1                         Rofl. Its true to its name  Non-Spam
1         2  The guy did some bitching but I acted like i'd...  Non-Spam
2         3  Pity, * was in mood for that. So...any other s...  Non-Spam
3         4               Will ü b going to esplanade fr home?  Non-Spam
4         5  This is the 2nd time we have tried 2 contact u...      Spam
..      ...                                                ...       ...
952     953  hows my favourite person today? r u workin har...  Non-Spam
953     954                      How much you got for cleaning  Non-Spam
954     955  Sorry da. I gone mad so many pending works wha...  Non-Spam
955     956                                 Wat time ü finish?  Non-Spam
956     957                    Just glad to be talking to you.  Non-Spam

[957 rows x 3 columns]

In [3]:
df.describe(include='object')

Message_body     Label
count                      957       957
unique                     945         2
top     Sorry, I'll call later  Non-Spam
freq                         4       835

In [4]:
df['input'] = df['Message_body']
df['label'] = pd.Categorical(df['Label'])
df['label'] = df['label'].cat.codes
df.input.head()

0                           Rofl. Its true to its name
1    The guy did some bitching but I acted like i'd...
2    Pity, * was in mood for that. So...any other s...
3                 Will ü b going to esplanade fr home?
4    This is the 2nd time we have tried 2 contact u...
Name: input, dtype: object

In [5]:
df['input']

0                             Rofl. Its true to its name
1      The guy did some bitching but I acted like i'd...
2      Pity, * was in mood for that. So...any other s...
3                   Will ü b going to esplanade fr home?
4      This is the 2nd time we have tried 2 contact u...
                             ...                        
952    hows my favourite person today? r u workin har...
953                        How much you got for cleaning
954    Sorry da. I gone mad so many pending works wha...
955                                   Wat time ü finish?
956                      Just glad to be talking to you.
Name: input, Length: 957, dtype: object

In [6]:
df = df.drop(columns = ['S. No.', 'Message_body', 'Label'], axis = 1)

In [7]:
# create dataset and tok_ds
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def tok_func(x):
    return tokenizer(x["input"])

tok_ds = ds.map(tok_func, batched = True)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/957 [00:00<?, ? examples/s]

In [8]:
tok_ds[0]

{'input': 'Rofl. Its true to its name',
 'label': 0,
 'input_ids': [1, 11288, 20516, 260, 2952, 980, 264, 359, 601, 2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
eval_df = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_test.csv", encoding='latin-1')
eval_df.describe(include = 'object')
eval_df['input'] = eval_df['Message_body']
eval_df['label'] = pd.Categorical(eval_df['Label'])
eval_df['label'] = eval_df['label'].cat.codes
eval_df = eval_df.drop(columns=['S. No.', 'Message_body', 'Label'], axis = 1)

eval_ds = Dataset.from_pandas(eval_df)
eval_ds = eval_ds.map(tok_func,batched = True)

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

In [10]:
dds = tok_ds.train_test_split(0.25)
dds

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 717
    })
    test: Dataset({
        features: ['input', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 240
    })
})

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

2024-07-11 17:50:26.622632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 17:50:26.622746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 17:50:26.748089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
import numpy as np

bs = 1
epochs = 1
lr = 8e-5



args = TrainingArguments('outputs', learning_rate=lr, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', fp16=True,
                         evaluation_strategy="epoch", 
                         per_device_train_batch_size=bs, 
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs, 
                         weight_decay=0.01, report_to='none')
   

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=2)
from sklearn.metrics import accuracy_score                                                         
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

learner  = Trainer(model, args, 
                   train_dataset=dds['train'], 
                   eval_dataset=dds['test'], 
                   tokenizer=tokenizer,
                   compute_metrics = compute_metrics  )     


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
learner.train()

TrainOutput(global_step=717, training_loss=0.6364544345743985, metrics={'train_runtime': 91.0502, 'train_samples_per_second': 7.875, 'train_steps_per_second': 7.875, 'total_flos': 8816947917372.0, 'train_loss': 0.6364544345743985, 'epoch': 1.0})